In [31]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


torch.manual_seed(1234)
np.random.seed(1234)

In [32]:
# load the data
X_train = pd.read_csv("../Data/X_train.csv")
y_train = pd.read_csv("../Data/y_train.csv")

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train.columns

X_train shape: (400, 9)
y_train shape: (400, 1)


Index(['User_ID', 'Age', 'Gender', 'Daily_Screen_Time(hrs)',
       'Sleep_Quality(1-10)', 'Stress_Level(1-10)',
       'Days_Without_Social_Media', 'Exercise_Frequency(week)',
       'Social_Media_Platform'],
      dtype='object')

In [33]:
# data preprocessing (drop ID, one hot encode categorical vars)

X_processed = X_train.drop(columns=["User_ID"])
X_processed = pd.get_dummies(X_processed, columns=["Gender", "Social_Media_Platform"], drop_first=True)
X_nn = X_processed.values
y_nn = y_train["Happiness_Index(1-10)"].values

print(f"X_nn shape: {X_nn.shape}")
print(f"y_nn shape: {y_nn.shape}")

X_nn shape: (400, 13)
y_nn shape: (400,)


In [34]:
# define neural network model

class NeuralNet(nn.Module):

    def __init__(self, input_dim=8, hidden_dim=16):
        super().__init__()
        l1 = nn.Linear(input_dim, hidden_dim)
        a1 = nn.ReLU()
        l2 = nn.Linear(hidden_dim, hidden_dim)
        a2 = nn.ReLU()
        l3 = nn.Linear(hidden_dim,1)
        l = [l1, a1, l2, a2, l3]
        self.module_list = nn.ModuleList(l)

    def forward(self, X):
        for f in self.module_list:
          X = f(X)
        return X

In [35]:
# set hyperparameter grid

params = {
    "hidden_dim": [16, 32],
    "lr": [0.001, 0.01],
    "batch_size": [16, 32]
}

In [41]:
# cross-validation and parameter tuning

k = 3
kf = KFold(n_splits = k, shuffle=True, random_state=1234)

results = []

for hidden_dim in params["hidden_dim"]:
    for lr in params["lr"]:
        for batch_size in params["batch_size"]:
            fold_mse = []
            fold_r2 = []

            print(f"\n--- Initializing Validation Loop. Params: hidden_dim={hidden_dim}, lr={lr}, batch_size={batch_size} ---\n")

            for fold, (train_index, val_index) in enumerate(kf.split(X_nn)):

                X_train, X_val = X_nn[train_index], X_nn[val_index]
                y_train, y_val = y_nn[train_index], y_nn[val_index]

                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)

                X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
                y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1,1)
                X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
                y_val_tensor = torch.tensor(y_val, dtype=torch.float32).reshape(-1,1)

                train_loader = DataLoader(
                    TensorDataset(X_train_tensor, y_train_tensor),
                    batch_size = batch_size,
                    shuffle = True
                )

                model = NeuralNet(input_dim=X_nn.shape[1], hidden_dim=hidden_dim)
                cost_function = nn.MSELoss()
                optimizer = optim.SGD(model.parameters(), lr=0.001)

                for epoch in range(100):
                    model.train()
                    for x_batch, y_batch in train_loader:
                        optimizer.zero_grad()
                        y_pred = model(x_batch)
                        cost = cost_function(y_pred, y_batch)
                        cost.backward()
                        optimizer.step()

                model.eval()
                with torch.no_grad():
                    y_pred_tensor = model(X_val_tensor)

                y_pred = y_pred_tensor.numpy().flatten()
                y_true = y_val_tensor.numpy().flatten()

                mse = mean_squared_error(y_true, y_pred)
                r2 = r2_score(y_true, y_pred)

                fold_mse.append(mse)
                fold_r2.append(r2)

                print(f"    Fold {fold+1} of {k} - Validation MSE: {mse:.4f}, Validation R²: {r2:.4f}")
            
            results.append({
               "hidden_dim": hidden_dim,
               "lr": lr,
                "batch_size": batch_size,
                "avg_mse": np.mean(fold_mse),
                "avg_r2": np.mean(fold_r2)
            })

            #print(f"  → Avg MSE: {np.mean(fold_mse):.4f}, Avg R²: {np.mean(fold_r2):.4f}")


            nn_average_mse = np.mean(fold_mse)
            nn_average_r2 = np.mean(fold_r2)

            print("\n   --- Overall Neural Network Cross-Validation Results ---")
            print(f"        Average MSE across {k} folds: {nn_average_mse:.4f}")
            print(f"        Average R² across {k} folds: {nn_average_r2:.4f}")
            


--- Initializing Validation Loop. Params: hidden_dim=16, lr=0.001, batch_size=16 ---

    Fold 1 of 3 - Validation MSE: 1.0440, Validation R²: 0.5327
    Fold 2 of 3 - Validation MSE: 1.3571, Validation R²: 0.5136
    Fold 3 of 3 - Validation MSE: 0.9268, Validation R²: 0.5596

   --- Overall Neural Network Cross-Validation Results ---
        Average MSE across 3 folds: 1.1093
        Average R² across 3 folds: 0.5353

--- Initializing Validation Loop. Params: hidden_dim=16, lr=0.001, batch_size=32 ---

    Fold 1 of 3 - Validation MSE: 0.9133, Validation R²: 0.5912
    Fold 2 of 3 - Validation MSE: 1.4090, Validation R²: 0.4950
    Fold 3 of 3 - Validation MSE: 1.2130, Validation R²: 0.4236

   --- Overall Neural Network Cross-Validation Results ---
        Average MSE across 3 folds: 1.1784
        Average R² across 3 folds: 0.5033

--- Initializing Validation Loop. Params: hidden_dim=16, lr=0.01, batch_size=16 ---

    Fold 1 of 3 - Validation MSE: 1.1011, Validation R²: 0.5072
  

In [43]:
results_df = pd.DataFrame(results)
results_df.sort_values("avg_mse")

,hidden_dim,lr,batch_size,avg_mse,avg_r2
0,16,0.001,16,1.109327,0.535290
2,16,0.010,16,1.128172,0.517125
1,16,0.001,32,1.178442,0.503264
6,32,0.010,16,1.199608,0.493955
3,16,0.010,32,1.228565,0.484258
4,32,0.001,16,1.258076,0.474315
7,32,0.010,32,1.305217,0.449838
5,32,0.001,32,1.306646,0.448249


In [44]:
results_df.sort_values("avg_r2")

,hidden_dim,lr,batch_size,avg_mse,avg_r2
5,32,0.001,32,1.306646,0.448249
7,32,0.010,32,1.305217,0.449838
4,32,0.001,16,1.258076,0.474315
3,16,0.010,32,1.228565,0.484258
6,32,0.010,16,1.199608,0.493955
1,16,0.001,32,1.178442,0.503264
2,16,0.010,16,1.128172,0.517125
0,16,0.001,16,1.109327,0.535290


In [46]:
best = results_df.loc[results_df["avg_mse"].idxmin()]
print("Best Params:")
print(best)

Best Params:
hidden_dim    16.000000
lr             0.001000
batch_size    16.000000
avg_mse        1.109327
avg_r2         0.535290
Name: 0, dtype: float64


In [49]:
# save for final eval notebook:

best_params = best.to_dict()

best_params_df = pd.DataFrame([best_params])
best_params_df.to_csv("./best_params/best_nn_params.csv", index=False)